# 要約 
このJupyterノートブックは、Kaggleの「LMSYS - Chatbot Arena」コンペティションにおいて、ユーザーの好みを予測するためのモデルを構築する過程を示しています。特に、異なるチャットボットモデルの応答の評価を基に、どちらがより好まれるかを予測するタスクに取り組んでいます。

### 問題とデータセット
- コンペティションは、ユーザーが異なる応答を選択する際の選好を予測することを目指しています。このノートブックでは、与えられたデータセットを用いて、どちらのモデルが勝者かを予測するための特徴量を生成し、分類モデルを訓練します。

### 手法とライブラリ
1. **データ前処理**:
   - `textstat`, `nltk`, `pandas`を使用して、テキストデータを読み込み、処理を行っています。具体的には、テキストから得られる単語数、文字数、文数、平均単語長、平均文長などの特徴量を計算しています。

2. **特徴量生成**:
   - 各応答の読みやすさスコアや頻度情報も計算され、最終的に訓練データセットに特徴量として追加されます。

3. **モデル構築**:
   - モデルとして、**Gradient Boosting**および**XGBoost**が使用されており、Scikit-learnの`RandomizedSearchCV`を用いてハイパーパラメータのチューニングが行われています。
   - 層化K分割交差検証を用いてモデルのパフォーマンスを評価し、ログ損失（log loss）をスコアとして用いて最良のモデルを選択します。

4. **予測と提出準備**:
   - 最終的に選ばれたモデルを用いてテストデータに対し予測を行い、結果を指定された提出形式（`submission.csv`）で出力します。

このノートブック全体を通じて、データの準備から特徴量の生成、モデルの訓練、予測および提出ファイルの生成まで一連の流れが分かりやすく実装されており、効果的な機械学習パイプラインが示されています。

---


# 用語概説 
以下に、Jupyter Notebookの内容をもとに、機械学習・深層学習において初心者がつまずく可能性のある専門用語の解説を行います。

1. **Pyphen**:
   - テキストのハイフネーション（単語の分割においてハイフンを使用する方法）を行うライブラリです。自然言語処理の際に、単語を適切に行分けするために使用されることがあります。

2. **textstat**:
   - テキストの統計情報を計算するためのライブラリで、文章の読みやすさや単語数、文の数などを評価する機能を提供します。言語の特性を評価するのに役立ちます。

3. **Gradient Boosting**:
   - 決定木を基本のアルゴリズムとし、予測性能を高めるために弱い学習器を徐々に組み合わせて強い学習器を作成するアンサンブル学習の手法です。特に、モデルの過去の予測の誤差を学ぶことで改善を図ります。

4. **XGBoost**:
   - Gradient Boostingの一種で、効率性と予測精度に特化した実装です。大規模なデータセットでも扱いやすく、競技プログラミングや実務でも広く利用されています。

5. **log_loss**:
   - ログ損失関数は、確率的予測の精度を測るための指標で、モデルが予測した確率と実際のクラスのラベルとの間の違いを評価します。数値が小さいほどモデルの性能が良いことを示します。

6. **StratifiedKFold**:
   - データセットを分割する際に、各クラスの比率が保たれるように層化して分割する方法です。特に不均衡なデータにおいて、モデル評価のバイアスを減らすのに役立ちます。

7. **RandomizedSearchCV**:
   - ハイパーパラメータのチューニングのための手法で、指定されたパラメータ空間からのランダムサンプリングを行い、最適なパラメータの組み合わせを見つけるために使われます。計算コストを抑えつつ、高速に探索を行うことができます。

8. **Counter**:
   - Pythonの標準ライブラリcollectionsにあるクラスで、要素のカウント機能を提供します。例えば、テキスト中の単語の頻度を簡単に計算するのに使用します。

9. **バイグラム (Bigram)**:
   - 連続する2つの単語の組み合わせのことを指します。言語モデルやテキスト解析で文脈を理解するための重要な特徴として用いられます。

10. **Flesch-Kincaid Score**:
    - 文章の難易度を評価するための指標で、主に米国の教育現場で利用されます。高いスコアは読みやすい文章を示し、低いスコアは難解な文章を示します。

11. **smog index**:
    - テキストの読みやすさを評価する指標で、特に教育や健康関連の文書において推奨される学年レベルを示します。単語の数と難しい単語の数から算出されます。

12. **型トークン比 (TTR)**:
    - テキスト内のユニークな単語の数を、総単語数で割った値です。言語の多様性を示す指標で、テキストがどれだけ異なる単語を使用しているかを評価します。

これらの用語を理解することで、ノートブックの内容がより深く把握でき、機械学習と自然言語処理の実践に役立つでしょう。

---


In [ ]:
!pip install ../input/textstat/Pyphen-0.10.0-py3-none-any.whl  # Pyphenライブラリをインストール
!pip install ../input/textstat/textstat-0.7.0-py3-none-any.whl  # textstatライブラリをインストール
import textstat  # textstatをインポートして、テキスト統計に関する機能を使用できるようにする

In [ ]:
import sklearn  # sklearnをインポート
import numpy as np  # numpyをインポート（数値計算のためのライブラリ）
import pandas as pd  # pandasをインポート（データ操作のためのライブラリ）
import matplotlib.pyplot as plt  # matplotlibをインポート（グラフ描画のためのライブラリ）
import time  # timeをインポート（時間計測のために使用）
from xgboost import XGBClassifier  # XGBoostの分類器をインポート
from sklearn.ensemble import GradientBoostingClassifier  # 勾配ブースティングの分類器をインポート
from sklearn.metrics import log_loss  # ログ損失を評価指標としてインポート
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV  # データ分割と交差検証を行うための関数をインポート
import nltk  # nltkをインポート（自然言語処理ライブラリ）
import textstat  # textstatを再インポート（重複していますが、他の機能を使う可能性があるため）
from textblob import TextBlob  # テキスト処理のためのTextBlobをインポート
from collections import Counter  # 要素のカウントを行うためのCounterをインポート

import warnings  # 警告を管理するためのライブラリをインポート
warnings.filterwarnings("ignore")  # 警告を無視する設定
warnings.filterwarnings('ignore')  # 再度同じ設定（冗長）
pd.options.display.float_format = '{:.2f}'.format  # pandasの浮動小数点数の表示形式を設定
pd.set_option('display.max_rows', None)  # 行数を制限せずに表示設定
pd.set_option('display.max_columns', None)  # 列数を制限せずに表示設定

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータを読み込む
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出ファイルを読み込む

In [ ]:
train.head()  # トレーニングデータの最初の5行を表示する

In [ ]:
test.head()  # テストデータの最初の5行を表示する

In [ ]:
print(f"The size of the train data: {train.shape} is and the test data is: {test.shape}")  # トレーニングデータとテストデータのサイズを表示する

In [ ]:
print(train['winner_model_a'].value_counts())  # winner_model_aの値のカウントを表示
print(train['winner_model_b'].value_counts())  # winner_model_bの値のカウントを表示
print(train['winner_tie'].value_counts())  # winner_tieの値のカウントを表示

In [ ]:
# 図と軸を作成する
fig, axes = plt.subplots(3, 1, figsize=(7, 6))

# プロットする列を指定
columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

# 0と1の色を定義
colors = {0: 'steelblue', 1: 'salmon'}

# 各列をそれぞれのサブプロットにプロット
for i, column in enumerate(columns):
    ax = axes[i]
    value_counts = train[column].value_counts().sort_index()
    
    # 指定した色とラベルでバーをプロット
    bars = ax.bar(value_counts.index.astype(str), value_counts, color=[colors[idx] for idx in value_counts.index],
                  label=value_counts.index.map({0: 'Lose (0)', 1: 'Win (1)'}))
    
    # バーにカウントを注釈する
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3ポイントの垂直オフセット
                    textcoords="offset points",
                    ha='center', va='bottom')
    
    ax.set_xlabel('Winner')  # x軸のラベル
    ax.set_ylabel('Count')  # y軸のラベル
    ax.set_title(f'Model {column.split("_")[-1].capitalize()} Counts')  # タイトル設定
    ax.legend(title='Outcome', loc='upper right')  # 凡例を設定

# 全体のタイトルを追加し、レイアウトを調整
fig.suptitle('Distribution of Winners Across Models', fontsize=16)  # 図全体のタイトル
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # レイアウトを調整

# プロットを表示
plt.show()

In [ ]:
def process(input_str):  # 入力された文字列を処理する関数
    stripped_str = input_str.strip('[]')  # 先頭と末尾の[]を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文を分割して不要な""を削除
    return  ' '.join(sentences)  # 文を結合して返す

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # テストデータのprompt列を処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # テストデータのresponse_a列を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # テストデータのresponse_b列を処理

train.loc[:, 'prompt'] = train['prompt'].apply(process)  # トレーニングデータのprompt列を処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # トレーニングデータのresponse_a列を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # トレーニングデータのresponse_b列を処理

In [ ]:
train.head(3)  # トレーニングデータの最初の3行を表示

In [ ]:
test.head(3)  # テストデータの最初の3行を表示

In [ ]:
%%time  # 次の処理にかかる時間を計測するマジックコマンド

# 単語数を計算する関数
def word_count(text):
    return len(nltk.word_tokenize(text))  # テキストをトークンに分割し、単語数を返す

# 文字数を計算する関数
def char_count(text):
    return len(text)  # テキストの文字数を返す

# 文の数を計算する関数
def sentence_count(text):
    return len(nltk.sent_tokenize(text))  # テキストを文に分割し、文の数を返す

# 平均単語長を計算する関数
def avg_word_length(text):
    words = nltk.word_tokenize(text)  # テキストをトークンに分割
    if len(words) == 0:  # 単語がない場合
        return 0  # 0を返す
    return sum(len(word) for word in words) / len(words)  # 単語の長さの合計を単語数で割る

# 平均文長を計算する関数
def avg_sentence_length(text):
    words = nltk.word_tokenize(text)  # テキストをトークンに分割
    sentences = nltk.sent_tokenize(text)  # テキストを文に分割
    if len(sentences) == 0:  # 文がない場合
        return 0  # 0を返す
    return len(words) / len(sentences)  # 単語数を文の数で割る

# 型トークン比を計算する関数
def ttr(text):
    words = nltk.word_tokenize(text)  # テキストをトークンに分割
    if len(words) == 0:  # 単語がない場合
        return 0  # 0を返す
    unique_words = set(words)  # ユニークな単語を取得
    return len(unique_words) / len(words)  # ユニーク単語数を全単語数で割る

# 単語頻度を計算する関数
def word_freq(text):
    words = nltk.word_tokenize(text)  # テキストをトークンに分割
    return Counter(words)  # 単語のカウントを返す

# バイグラム頻度を計算する関数
def bigram_freq(text):
    words = nltk.word_tokenize(text)  # テキストをトークンに分割
    bigrams = list(nltk.bigrams(words))  # バイグラムを取得
    return Counter(bigrams)  # バイグラムのカウントを返す

# 読みやすさスコアを計算する関数
def readability_scores(text):
    scores = {  # 各種読みやすさスコアを計算
        "flesch_kincaid_score": textstat.flesch_kincaid_grade(text),
        "gunning_fog_index": textstat.gunning_fog(text),
        "smog_index": textstat.smog_index(text),
        "ari": textstat.automated_readability_index(text)
    }
    return scores  # スコア辞書を返す

# 追加のメトリックを計算し、データフレームに追加
for column in ["prompt", "response_a", "response_b"]:
    train[f"{column}_word_count"] = train[column].apply(word_count)  # 各列に単語数を追加
    train[f"{column}_char_count"] = train[column].apply(char_count)  # 各列に文字数を追加
    train[f"{column}_sentence_count"] = train[column].apply(sentence_count)  # 各列に文の数を追加
    train[f"{column}_avg_word_length"] = train[column].apply(avg_word_length)  # 各列に平均単語長を追加
    train[f"{column}_avg_sentence_length"] = train[column].apply(avg_sentence_length)  # 各列に平均文長を追加
#     train[f"{column}_ttr"] = train[column].apply(ttr)  # 型トークン比を計算して追加（コメントアウト）
#     readability = train[column].apply(readability_scores)  # 読みやすさスコアを計算
#     train[f"{column}_flesch_kincaid_score"] = readability.apply(lambda x: x["flesch_kincaid_score"])  # Flesch-Kincaidスコアを追加
#     train[f"{column}_gunning_fog_index"] = readability.apply(lambda x: x["gunning_fog_index"])  # Gunning-Fogインデックスを追加
#     train[f"{column}_smog_index"] = readability.apply(lambda x: x["smog_index"])  # SMOGインデックスを追加
#     train[f"{column}_ari"] = readability.apply(lambda x: x["ari"])  # ARIスコアを追加

train.head()  # トレーニングデータの最初の5行を表示

In [ ]:
%%time  # 次の処理にかかる時間を計測するマジックコマンド

import time  # timeを再インポート（重複していますが、他の機能を使う可能性があるため）
from sklearn.ensemble import GradientBoostingClassifier  # 勾配ブースティングの分類器を再インポート（重複していますが、他の機能を使う可能性があるため）
from xgboost import XGBClassifier  # XGBoostの分類器を再インポート（重複していますが、他の機能を使う可能性があるため）
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV  # データ分割と交差検証を行うための関数を再インポート（重複していますが、他の機能を使う可能性があるため）
from sklearn.metrics import log_loss  # ログ損失を評価指標として再インポート（重複していますが、他の機能を使う可能性があるため）
from scipy.stats import uniform, randint  # scipyから乱数生成のためのライブラリをインポート

# ターゲットをカテゴリカルラベルの単一列に変換
train['winner'] = (train['winner_model_a'] * 1 + train['winner_model_b'] * 2 + train['winner_tie'] * 3).astype(int)

# 特徴量とターゲットを定義
columns_to_remove = {'id', 'model_a', 'model_b', 'prompt', 'response_a', 'response_b', 
                     'winner_model_a', 'winner_model_b', 'winner_tie', 'winner'}

features = [col for col in train.columns if col not in columns_to_remove]  # 特徴量のリストを作成

X = train[features]  # 特徴量データをXに設定
y = train['winner'] - 1  # ターゲットデータをyに設定

# データをトレーニングセットとバリデーションセットに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# モデルを定義
models = {
    'GradientBoostingClassifier': GradientBoostingClassifier(),  # 勾配ブースティング分類器
    'XGBClassifier': XGBClassifier()  # XGBoost分類器
}

# ランダムサーチのためのパラメータの分布を定義
param_distributions = {
    'GradientBoostingClassifier': {
        'n_estimators': [100,200,350,300],  # 使用する決定木の数
        'max_depth': [2,3,4,5,7,9]  # 決定木の最大深さ
    },
    'XGBClassifier': {
        'n_estimators': [100,200,350,300],  # 使用する決定木の数
        'max_depth': [2,3,4,5,7,9]  # 決定木の最大深さ
    }
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234)  # 層化K分割交差検証を設定

best_models = {}  # 最良モデルを保存する辞書

# 各モデルに対して繰り返し
for model_name, model in models.items():
    print(f"Model training for {model_name}")  # 現在のモデル名を表示
    
    # ランダムサーチの実行
    random_search = RandomizedSearchCV(model, param_distributions[model_name], n_iter=10, scoring='neg_log_loss', 
                                       n_jobs=-1, cv=skf, random_state=42)
    random_search.fit(X_train, y_train)  # トレーニングデータでモデルをフィッティング
    
    best_model = random_search.best_estimator_  # ベストモデルを取得
    best_models[model_name] = best_model  # 現在のモデルタイプのベストモデルを保存
    
    logloss_scores = []  # ログ損失スコアを保存するリスト
    start_time = time.time()  # 計測開始時間
    
    count = 0  # カウント初期化
    for train_index, test_index in skf.split(X, y):  # K分割交差検証のための分割
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]  # トレーニングフォールドとテストフォールド
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]  # ターゲットのフォールド

        best_model.fit(X_train_fold, y_train_fold)  # 最良モデルでトレーニングフォールドをフィッティング
        y_test_pred_proba = best_model.predict_proba(X_test_fold)  # テストフォールドに対する確率を予測

        logloss = log_loss(y_test_fold, y_test_pred_proba)  # ログ損失を計算
        logloss_scores.append(logloss)  # スコアをリストに追加
        print(f"The log loss score for fold {count}: {logloss}")  # 現在のフォールドのログ損失を表示
        count += 1  # カウントをインクリメント

    average_logloss = sum(logloss_scores) / len(logloss_scores)  # 平均ログ損失を計算
    print(f"The average log loss score for {model_name} across all folds: {average_logloss}")  # すべてのフォールドの平均ログ損失を表示
    
    elapsed_time = time.time() - start_time  # 経過時間を計測
    print(f"Time taken for {model_name}: {elapsed_time:.2f} seconds")  # モデルのトレーニングにかかった時間を表示
    
    # バリデーションセットに対する確率を予測
    y_val_prob = best_model.predict_proba(X_val)
    # バリデーションセットでのログ損失を計算
    val_loss = log_loss(y_val, y_val_prob)
    print(f'Log Loss using {model_name} on validation set: {val_loss}')  # バリデーションセットでのログ損失を表示

# バリデーションセットのパフォーマンスに基づいてベストモデルを特定
best_model_name = min(best_models, key=lambda k: log_loss(y_val, best_models[k].predict_proba(X_val)))  # 最良モデル名を取得
best_average_logloss = log_loss(y_val, best_models[best_model_name].predict_proba(X_val))  # 最良モデルの平均ログ損失を計算

print(f"The best model is {best_model_name} with an average log loss score of {best_average_logloss}")  # 最良モデルとその平均ログ損失を表示

In [ ]:
model_to_use = best_models[best_model_name]  # 使用するモデルを最良モデルに設定
model_to_use  # モデル情報を表示

In [ ]:
# 追加のメトリックを計算し、データフレームに追加
for column in ["prompt", "response_a", "response_b"]:
    test[f"{column}_word_count"] = test[column].apply(word_count)  # 各列に単語数を追加
    test[f"{column}_char_count"] = test[column].apply(char_count)  # 各列に文字数を追加
    test[f"{column}_sentence_count"] = test[column].apply(sentence_count)  # 各列に文の数を追加
    test[f"{column}_avg_word_length"] = test[column].apply(avg_word_length)  # 各列に平均単語長を追加
    test[f"{column}_avg_sentence_length"] = test[column].apply(avg_sentence_length)  # 各列に平均文長を追加
    
test.head()  # テストデータの最初の5行を表示

In [ ]:
test_features = test[features]  # テストデータの特徴量を取得
test_predictions = model_to_use.predict_proba(test_features)  # テストデータに対する予測確率を計算

In [ ]:
test_predictions  # テストデータに対する予測確率を表示

In [ ]:
# 提出ファイルを準備する
submission = pd.DataFrame({  # データフレームを生成
    'id': test['id'],  # テストデータのIDを含める
    'winner_model_a': test_predictions[:, 0],  # モデルAの勝者確率
    'winner_model_b': test_predictions[:, 1],  # モデルBの勝者確率
    'winner_tie': test_predictions[:, 2]  # 引き分けの確率
})

In [ ]:
submission.head()  # 提出ファイルの最初の5行を表示

In [ ]:
submission.to_csv('/kaggle/working/submission.csv', index=False)  # 提出ファイルをCSV形式で保存